Stocks Rag

Author: Orvin Bellamy (https://github.com/orvinbellamy)

In [1]:
## Import libraries

import yfinance as yf
import pandas as pd
import openai
from openai import OpenAI
import os
import json
import fs
import time
from dotenv import load_dotenv
import logging as log
from matplotlib import pyplot as plt
from matplotlib import image as mpimg
from filehandler import FileHandler
from yfinancehandler import YFHandler
from eventhandler import EventHandler, ThreadManager
from agenthandler import AgentHandler
import importlib

In [2]:
## configruation

FILE_PATH = 'openai_upload_files/'
OPENAI_DIC_FILE_NAME = 'openai_files.json'

# Set constants for loops
NUM_TRIES = 12
TIME_SLEEP = 5 # in seconds

# Load schemas from JSON file
with open('config/dataframe_schemas.json', 'r') as f:
    schemas = json.load(f)

with open('config/config.json', 'r') as f:
    dic_config = json.load(f)
    OPEN_API_KEY = dic_config['OPEN_API_KEY']

# Initialize an empty list to store the stocks
stocks_list = []

# Open the text file in read mode
with open('stocks.txt', 'r') as file:
    # Read each line of the file
    for line in file:
        # Strip any leading or trailing whitespace and append the line to the list
        stocks_list.append(line.strip())

df_portfolio = pd.read_csv('config/portfolio.csv')

list_portfolio = list(df_portfolio['ticker'].unique())

# Open dic_files
with open(f'{FILE_PATH}{OPENAI_DIC_FILE_NAME}', 'r') as f:
    dic_files = json.load(f)

client = OpenAI(api_key=OPEN_API_KEY)

### Stocks Data

In [9]:
stocks_list = ['MSFT', 'SHOP', 'AMD', 'NVDA', 'AAPL', 'OKTA', 'SHOP', 'AMAT', 'TTWO', 'SNOW', 'SOXX', 'BA', 'DOCN', 'MCD', 'LULU', 'CSCO', 'ORCL', 'AMZN', 'ASML']

yf_handler = YFHandler(stock_list=stocks_list, schemas=schemas)

df_stocks = yf_handler.import_stocks()
df_cashflow = yf_handler.import_cashflow()
df_income_stmt = yf_handler.import_income_stmt()

# Write to CSV
# Technically this will be done by the FileHandler but just to be safe
df_stocks.to_csv('openai_upload_files/df_stocks.csv', index=False)
df_cashflow.to_csv('openai_upload_files/df_cashflow.csv', index=False)
df_income_stmt.to_csv('openai_upload_files/df_income_stmt.csv', index=False)

file_stocks = FileHandler(
    df=df_stocks,
    dic_file=dic_files,
    file_name='df_stocks.csv',
    dic_file_name=OPENAI_DIC_FILE_NAME,
    file_path=FILE_PATH,
    dic_file_path=FILE_PATH,
    client=client
)

file_cashflow = FileHandler(
    df=df_cashflow,
    dic_file=dic_files,
    file_name='df_cashflow.csv',
    dic_file_name=OPENAI_DIC_FILE_NAME,
    file_path=FILE_PATH,
    dic_file_path=FILE_PATH,
    client=client
)

file_income_stmt = FileHandler(
    df=df_income_stmt,
    dic_file=dic_files,
    file_name='df_income_stmt.csv',
    dic_file_name=OPENAI_DIC_FILE_NAME,
    file_path=FILE_PATH,
    dic_file_path=FILE_PATH,
    client=client
)

d:\Documents\Python Repo\stock_rag\yfinancehandler.py:20: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)
d:\Documents\Python Repo\stock_rag\yfinancehandler.py:34: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, df_plc], ignore_index=True)
d:\Documents\Python Repo\stock_rag\yfinancehandler.py:48: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longe

In [10]:
# Update files
file_stocks.update_openai_file(dic_file=dic_files)
file_cashflow.update_openai_file(dic_file=dic_files)
file_income_stmt.update_openai_file(dic_file=dic_files)

file name: df_stocks.csv, file id: file-DBk7T2O7LP5aCpTvtfSbXlwK has been deleted
file name: df_stocks.csv is uploaded, new file id: file-FigeHObs1GpRJW0eRD4yYmMK
openai_upload_files/openai_files.json file has been updated
file name: df_cashflow.csv, file id: file-DHZyBZkKJXv900f8xhwfk12P has been deleted
file name: df_cashflow.csv is uploaded, new file id: file-19mhjbyUZALVok1nO5cRtZ5m
openai_upload_files/openai_files.json file has been updated
file name: df_income_stmt.csv, file id: file-i4sIF7ncaAsQAENo4cJMpypJ has been deleted
file name: df_income_stmt.csv is uploaded, new file id: file-v9sPcDmxULrlvLuyyj90Mv8N
openai_upload_files/openai_files.json file has been updated


In [12]:
with open('config/assistants.json', 'r') as json_file:
    dic_assistants= json.load(json_file)

# Have to manually update the tool_resources because the file_id can change
dic_assistants['fin_analyst']['tool_resources'] = {
    'code_interpreter': {'file_ids': [dic_files['df_stocks.csv']]}
}

with open('config/assistants.json', 'w') as json_file:
    json.dump(dic_assistants, json_file)
    print(f'assistants.json file has been updated')

fin_analyst = AgentHandler(
    client = client, 
    new=False,
    assistant_name = 'fin_analyst',
    dic_file = dic_assistants,
    dic_file_name = 'assistants.json',
    dic_file_path='config/'
    )

fin_consultant = AgentHandler(
    client = client,
    new=False,
    assistant_name = 'fin_consultant',
    dic_file = dic_assistants,
    dic_file_name = 'assistants.json',
    dic_file_path='config/'
    )

assistants.json file has been updated


In [ ]:
def analyze_stock(client: OpenAI, ticker):
	
	yf_handler = YFHandler(stock_list=ticker, schemas=schemas)

	df_stocks = yf_handler.import_stocks()
	df_cashflow = yf_handler.import_cashflow()
	df_income_stmt = yf_handler.import_income_stmt()

	# Write to CSV
	# Technically this will be done by the FileHandler but just to be safe
	df_stocks.to_csv('openai_upload_files/df_stocks.csv', index=False)
	df_cashflow.to_csv('openai_upload_files/df_cashflow.csv', index=False)
	df_income_stmt.to_csv('openai_upload_files/df_income_stmt.csv', index=False)

	file_stocks = FileHandler(
		df=df_stocks,
		dic_file=dic_files,
		file_name='df_stocks.csv',
		dic_file_name=OPENAI_DIC_FILE_NAME,
		file_path=FILE_PATH,
		dic_file_path=FILE_PATH,
		client=client
	)

	file_cashflow = FileHandler(
		df=df_cashflow,
		dic_file=dic_files,
		file_name='df_cashflow.csv',
		dic_file_name=OPENAI_DIC_FILE_NAME,
		file_path=FILE_PATH,
		dic_file_path=FILE_PATH,
		client=client
	)

	file_income_stmt = FileHandler(
		df=df_income_stmt,
		dic_file=dic_files,
		file_name='df_income_stmt.csv',
		dic_file_name=OPENAI_DIC_FILE_NAME,
		file_path=FILE_PATH,
		dic_file_path=FILE_PATH,
		client=client
	)

	# Update files on OpenAI
	file_stocks.update_openai_file(dic_file=dic_files)
	file_cashflow.update_openai_file(dic_file=dic_files)
	file_income_stmt.update_openai_file(dic_file=dic_files)

	# Have to manually update the tool_resources because the file_id can change
	dic_assistants['fin_analyst']['tool_resources'] = {
		'code_interpreter': {'file_ids': [dic_files['df_stocks.csv']]}
	}

	with open('config/assistants.json', 'w') as json_file:
		json.dump(dic_assistants, json_file)
		print(f'assistants.json file has been updated')

	fin_analyst = AgentHandler(
		client = client, 
		new=False,
		assistant_name = 'fin_analyst',
		dic_file = dic_assistants,
		dic_file_name = 'assistants.json',
		dic_file_path='config/'
		)

	fin_consultant = AgentHandler(
		client = client,
		new=False,
		assistant_name = 'fin_consultant',
		dic_file = dic_assistants,
		dic_file_name = 'assistants.json',
		dic_file_path='config/'
		)

In [48]:
## Create thread
thread = ThreadManager(
    client=client,
    prompt='Can you explain to me what the file I have attached is.',
    attachments=['file-G8Scw9sLUsW5L6oyEAGb3O30']
)


In [49]:
thread.run_thread(assistant = fin_analyst)


assistant > code_interpreter

import pandas as pd

# Load the provided files to understand their content
file_1_path = '/mnt/data/file-G8Scw9sLUsW5L6oyEAGb3O30'
file_2_path = '/mnt/data/file-vxqqbOjZo86TSLYt5jWlQPqy'

# Try reading the files to understand their structure and content
file_1 = pd.read_csv(file_1_path)
file_1.head(), file_1.describe()
assistant > The first file contains historical stock market data, including various attributes such as:

1. **Date**: The date of the stock market data.
2. **Open**: The opening price of the stock on that day.
3. **High**: The highest price at which the stock traded on that day.
4. **Low**: The lowest price at which the stock traded on that day.
5. **Close**: The closing price of the stock on that day.
6. **Volume**: The number of shares traded on that day.
7. **Dividends**: The dividend paid by the stock on that day.
8. **Stock Splits**: Any stock splits that occurred on that day.
9. **Ticker**: The stock's ticker symbol.
10. **Capital Gai

In [51]:
prompt = [
	{
		'role': 'user',
		'content': 'This is your finance consultant. Give me your in-depth analysis of Microsoft (MSFT) stock.',
		'attachments': [
			{
				'file_id': 'file-G8Scw9sLUsW5L6oyEAGb3O30',
                'tools': [{'type': 'code_interpreter'}]
			}
		]
	}
]

thread.run_thread(assistant=fin_analyst, prompt=prompt)


assistant > Great. Let's perform an in-depth analysis of Microsoft (MSFT) stock using the provided historical data. We'll focus on several key aspects:

1. **Descriptive Analysis**: Understanding basic statistics like mean, median, standard deviation for stock prices (open, high, low, close) and volume.
2. **Trend Analysis**: Visualizing the trend of the stock prices over the last 6 months.
3. **Volatility Analysis**: Analyzing the stock's volatility over the last 6 months.
4. **Volume Analysis**: Examining the stock's trading volume over the last 6 months.
5. **Moving Averages**: Calculating and plotting moving averages to understand the short-term and long-term trends.
6. **Technical Indicators**: Using indicators like RSI (Relative Strength Index) and MACD (Moving Average Convergence Divergence) for a more detailed technical analysis.

First, let’s filter the data for Microsoft (MSFT) and focus on the last 6 months. We'll then proceed with the subsequent analyses.
assistant > code_

In [57]:
prompt = [
	{
		'role': 'user',
		'content': f'Below is an analysis of the Microsoft (MSFT) stock done by your financial analyst. Use this to provide a recommendation to your client who is currently holding MSFT stock worth $10,000 and currently has a ROI of -10%.\n {fin_analyst_analysis}'
	}
]

thread.run_thread(assistant=fin_consultant, prompt=prompt)


assistant > ### Recommendation for Microsoft (MSFT) Stock Holding

**Dear Client,**

Based on the recent in-depth analysis of Microsoft's (MSFT) stock, here are the findings and my corresponding recommendation for your investment:

### Analysis Summary:
1. **Technical Indicators Analysis:**
   - **RSI (Relative Strength Index):** The RSI has stayed within the range of 30 to 70, suggesting that the stock isn’t extremely overbought or oversold. This implies a balanced momentum.
   - **MACD (Moving Average Convergence Divergence):** The MACD has shown some bullish crossover events over the last 6 months, indicating potential upward momentum. The MACD line crossing above the Signal line and a positive histogram supports this bullish signal.

2. **Trend and Volatility Analysis:**
   - **Moving Averages:** The 20-day SMA (Simple Moving Average) being above the 50-day SMA indicates a strong short-term momentum. Recent crossover of the 20-day SMA above the 50-day SMA is a bullish signal.
   -

In [61]:
print(thread.dic_thread['msg_ouARlzkl6WjYFABVNjPuru1l']['message_text'])
# messages = client.beta.threads.messages.list(thread_id=thread.thread_id)
		
# # Get the list of messages in the thread
# messages_data = messages.data

# for message in messages_data:
#     print(message.id)
# file_id = 'file-ZGJobD95f0QsOKv03l2dTtUW'
# file = client.files.content(file_id)
# thread.get_last_message()

### Recommendation for Microsoft (MSFT) Stock Holding

**Dear Client,**

Based on the recent in-depth analysis of Microsoft's (MSFT) stock, here are the findings and my corresponding recommendation for your investment:

### Analysis Summary:
1. **Technical Indicators Analysis:**
   - **RSI (Relative Strength Index):** The RSI has stayed within the range of 30 to 70, suggesting that the stock isn’t extremely overbought or oversold. This implies a balanced momentum.
   - **MACD (Moving Average Convergence Divergence):** The MACD has shown some bullish crossover events over the last 6 months, indicating potential upward momentum. The MACD line crossing above the Signal line and a positive histogram supports this bullish signal.

2. **Trend and Volatility Analysis:**
   - **Moving Averages:** The 20-day SMA (Simple Moving Average) being above the 50-day SMA indicates a strong short-term momentum. Recent crossover of the 20-day SMA above the 50-day SMA is a bullish signal.
   - **Volatility

In [62]:
thread.delete_thread()

thread: thread_Ab1FHM2gH5IphWS4rf3hhBsp has been deleted.
